In [1]:
import pandas as pd
import os
from openai import OpenAI
import openai
from dotenv import load_dotenv, find_dotenv
from qdrant_client import models, QdrantClient
from qdrant_client.http import models as rest
from qdrant_client.http.models import Record
from sentence_transformers import SentenceTransformer

load_dotenv(find_dotenv())
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
)

# Read the CSV dataset
file_path = '../datasets/songs_embeddings.csv'
df = pd.read_csv(file_path) # Add .head(100) if you want to limit the number of rows

df = df.fillna('')

print(str(len(df)) + ' rows')
df.head()

2222 rows


,id,artist,song,lyrics,img_src,sentiment_score,metadata,metadata_vector
0,0,米津玄師,Lemon,夢ならばどれほどよかったでしょう未だにあなたのことを夢にみる忘れた物を取りに帰るように古びた...,https://m.media-amazon.com/images/I/51ZsVIMARh...,-0.669566,米津玄師 / Lemon / 夢ならばどれほどよかったでしょう未だにあなたのことを夢にみる忘...,"[0.0367952436208725, -0.01923556998372078, -0...."
1,1,back number,クリスマスソング,どこかで鐘が鳴ってらしくない言葉が浮かんで寒さが心地よくてあれ なんで恋なんかしてんだろう聖...,https://m.media-amazon.com/images/I/31eRU7YYby...,-0.066641,back number / クリスマスソング / どこかで鐘が鳴ってらしくない言葉が浮かんで...,"[0.004695550538599491, -0.05602886155247688, -..."
2,2,GReeeeN,キセキ,明日、今日よりも好きになれる 溢れる想いが止まらない今もこんなに好きでいるのに 言葉に出来な...,https://m.media-amazon.com/images/I/51L0WT553N...,0.651807,GReeeeN / キセキ / 明日、今日よりも好きになれる 溢れる想いが止まらない今もこん...,"[0.0385475717484951, -0.042898621410131454, 0...."
3,3,back number,花束,どう思う？これから2人でやっていけると思う？んんどうかなぁでもとりあえずは一緒にいたいと思っ...,https://m.media-amazon.com/images/I/51SHzjh0dC...,-0.711921,back number / 花束 / どう思う？これから2人でやっていけると思う？んんどうか...,"[-0.005037600640207529, -0.05399758741259575, ..."
4,4,RADWIMPS,前前前世 (movie ver.),やっと眼を覚ましたかい それなのになぜ眼も合わせやしないんだい？「遅いよ」と怒る君 これでも...,https://m.media-amazon.com/images/I/51h10DfD1o...,-0.626399,RADWIMPS / 前前前世 (movie ver.) / やっと眼を覚ましたかい それな...,"[0.03341161459684372, 0.004328139126300812, -0..."


In [2]:
import ast

# Initialize the Qdrant client
qdrant_client = QdrantClient(':memory:')

# Set the collection name and size
collection_name = 'songs'
vector_size = len(ast.literal_eval(df['metadata_vector'][0]))  # Convert string to list and get its length

# Create a collection
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config={
        'metadata': rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
    }
)

# Add vectors to the collection
qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        rest.PointStruct(
            id=k,
            vector={
                'metadata': ast.literal_eval(v['metadata_vector']),  # Convert string to list
            },
            payload=v.to_dict(),
        )
        for k, v in df.iterrows()
    ],
)

print(qdrant_client.get_collections())
qdrant_client.count(collection_name=collection_name)

collections=[CollectionDescription(name='songs')]


CountResult(count=2222)

In [6]:
# Generate a query embedding and search in Qdrant
def query_qdrant(query, collection_name, vector_name, top_k=20):
    # Creates embedding vector from user query
    completion = openai.embeddings.create(
        input=query,
        model='text-embedding-3-small'  # Be sure to use the same embedding model as the vectors in the collection
    )

    embedded_query = completion.data[0].embedding

    query_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=(
            vector_name, embedded_query
        ),
        limit=top_k,
    )
    
    return query_results

In [12]:
# Search for similar vectors and store a result
query_results = query_qdrant('高校生時代の恋に関する曲', collection_name=collection_name, vector_name='metadata')

for i, vector in enumerate(query_results):
    print(f"{i + 1}. {vector.payload['song']} {vector.payload['artist']} (Score: {round(vector.score, 3)})")

1. 高校三年生 舟木一夫 (Score: 0.521)
2. 初恋 村下孝蔵 (Score: 0.502)
3. LOVE SONG CHAGE and ASKA (Score: 0.499)
4. 学生時代 ペギー葉山 (Score: 0.487)
5. 純恋歌 湘南乃風 (Score: 0.483)
6. Luv Bias Kis-My-Ft2 (Score: 0.468)
7. 青春時代 森田公一とトップギャラン (Score: 0.466)
8. Highschool love E-girls (Score: 0.466)
9. Gee 少女時代 (Score: 0.462)
10. Love Story 安室奈美恵 (Score: 0.459)
11. Sweet Tweet Official髭男dism (Score: 0.459)
12. 純愛ラプソディ 竹内まりや (Score: 0.458)
13. プルメリア ～花唄～ Aqua Timez (Score: 0.456)
14. 夏の日の1993 class (Score: 0.448)
15. 白い恋人達 桑田佳祐 (Score: 0.448)
16. 青いベンチ サスケ (Score: 0.447)
17. 君はロックを聴かない あいみょん (Score: 0.445)
18. 小さな恋のうた MONGOL800 (Score: 0.443)
19. ずっと2人で… GLAY (Score: 0.44)
20. また君に恋してる 坂本冬美 (Score: 0.44)


In [24]:
# Search for similar vectors and store a result
query_results = query_qdrant('海に関する', collection_name=collection_name, vector_name='metadata')

for i, vector in enumerate(query_results):
    print(f"{i + 1}. {vector.payload['song']} {vector.payload['artist']} (Score: {round(vector.score, 3)})")

1. 宙船(そらふね) TOKIO (Score: 0.347)
2. 海の声 BEGIN (Score: 0.328)
3. Friend Ship 星野源 (Score: 0.325)
4. OCEAN B'z (Score: 0.322)
5. 海の声 浦島太郎(桐谷健太) (Score: 0.32)
6. 海の幽霊 米津玄師 (Score: 0.317)
7. 風のゆくえ Ado (Score: 0.314)
8. 島人ぬ宝 BEGIN (Score: 0.313)
9. ビンクスの酒 麦わらの一味 (Score: 0.309)
10. RIP Orangestar (Score: 0.308)
11. Hey Ho SEKAI NO OWARI (Score: 0.292)
12. みだれ髪 美空ひばり (Score: 0.29)
13. 別れの予感 テレサ・テン (Score: 0.285)
14. RPG SEKAI NO OWARI (Score: 0.284)
15. ガールズルール 乃木坂46 (Score: 0.278)
16. 瑠璃色の地球 松田聖子 (Score: 0.273)
17. 世界中のこどもたちが 童謡・唱歌 (Score: 0.272)
18. シスター ポルノグラフィティ (Score: 0.269)
19. ドラえもんえかきうた 大山のぶ代 (Score: 0.268)
20. ファイト！ 中島みゆき (Score: 0.261)


In [23]:
# Search for similar vectors and store a result
query_results = query_qdrant('懐かしい', collection_name=collection_name, vector_name='metadata')

for i, vector in enumerate(query_results):
    print(f"{i + 1}. {vector.payload['song']} {vector.payload['artist']} (Score: {round(vector.score, 3)})")

1. 駅 竹内まりや (Score: 0.426)
2. SWEET MEMORIES 松田聖子 (Score: 0.422)
3. 想い出がいっぱい H2O (Score: 0.421)
4. ら・ら・ら 大黒摩季 (Score: 0.416)
5. やさしさに包まれたなら 松任谷由実 (Score: 0.407)
6. カブトムシ aiko (Score: 0.405)
7. ロード THE 虎舞竜 (Score: 0.405)
8. 二人 aiko (Score: 0.4)
9. うたかた花火 supercell (Score: 0.398)
10. 帰り道は遠回りしたくなる 乃木坂46 (Score: 0.398)
11. 花鳥風月 SEKAI NO OWARI (Score: 0.395)
12. 優しい彗星 YOASOBI (Score: 0.394)
13. ドーナツホール 米津玄師 (Score: 0.393)
14. 勝手にシンドバッド サザンオールスターズ (Score: 0.393)
15. 愛燦燦 美空ひばり (Score: 0.392)
16. 蕾 コブクロ (Score: 0.391)
17. Pieces L'Arc～en～Ciel (Score: 0.39)
18. また今日と同じ明日が来る 二宮和也(嵐) (Score: 0.389)
19. うれしい！たのしい！大好き！ DREAMS COME TRUE (Score: 0.389)
20. 夢見月に何想ふ RADWIMPS (Score: 0.387)


In [28]:
# Search for similar vectors and store a result
query_results = query_qdrant('果物', collection_name=collection_name, vector_name='metadata')

for i, vector in enumerate(query_results):
    print(f"{i + 1}. {vector.payload['song']} {vector.payload['artist']} (Score: {round(vector.score, 3)})")

1. アップルパイ back number (Score: 0.337)
2. Flamingo 米津玄師 (Score: 0.317)
3. いろはにほへと 椎名林檎 (Score: 0.315)
4. 種のうた ストロベリー・フラワー (Score: 0.312)
5. あめふりくまのこ おかあさんといっしょ (Score: 0.308)
6. 真夏の果実 サザンオールスターズ (Score: 0.304)
7. おべんとうばこのうた 童謡・唱歌 (Score: 0.301)
8. あんずのうた 双葉杏(五十嵐裕美) (Score: 0.299)
9. Lemon 米津玄師 (Score: 0.298)
10. CANDY Mr.Children (Score: 0.297)
11. 愛のうた ストロベリー・フラワー (Score: 0.291)
12. ミックスナッツ Official髭男dism (Score: 0.289)
13. 勇気りんりん ドリーミング (Score: 0.288)
14. 紅葉(もみじ) 童謡・唱歌 (Score: 0.284)
15. ちょうちょう 童謡・唱歌 (Score: 0.282)
16. リンゴの唄 並木路子 (Score: 0.275)
17. ありあまる富 椎名林檎 (Score: 0.274)
18. 愛のままで… 秋元順子 (Score: 0.274)
19. PEACH 大塚愛 (Score: 0.274)
20. 黒毛和牛上塩タン焼680円 大塚愛 (Score: 0.27)
